In [21]:
import cv2 as cv
import sys
import numpy as np
import matplotlib.pyplot as plt
import os

In [22]:
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_01.jpg"
# img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/imagini_auxiliare/02.jpg"
img1 = cv.imread(img_path)
img_path =  "C:/Programs/facultate/CAVA/CAVA-2024-Tema1/antrenare/1_02.jpg"
img2 = cv.imread(img_path)

### Use hsv colors to diferentiate the board from the rest of the background and get a mask

In [23]:
def crop_image_hsv(img):
    # img = cv.imread(img_path)
    # img=cv.resize(img,(0,0),fx=0.20,fy=0.20)
    # low_yellow = (14, 0, 0)
    # high_yellow = (255, 255, 255)
    # low_yellow = (0, 0, 0)
    # high_yellow = (14, 255, 255)
    low_yellow = (0, 110, 0)
    high_yellow = (255, 255, 255)
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    mask_yellow_hsv = cv.inRange(img_hsv, low_yellow, high_yellow)

    kernel = np.ones((3,3),np.uint8)
    mask_yellow_hsv = cv.morphologyEx(mask_yellow_hsv, cv.MORPH_OPEN, kernel)
    # cv.imshow("graay",mask_yellow_hsv)
    # image_bi_blur = cv.bilateralFilter(img, 15, 75, 75) 
    # gray = cv.cvtColor(image_bi_blur,cv.COLOR_BGR2GRAY)
    # print(mask_yellow_hsv.shape)
    # print(gray.shape)
    # mask_yellow_hsv = cv.bitwise_not(mask_yellow_hsv)
    # pre = cv.bitwise_and(gray,gray,mask=mask_yellow_hsv)
    # crop_img=cv.resize(crop_img,(0,0),fx=0.20,fy=0.20)
    # crop_img = extrage_careu(crop_img)
    # crop_img = cv.cvtColor(crop_img,cv.COLOR_BAYER_BG2GRAY)
    return mask_yellow_hsv

### Using the mask from the above function return only the board

In [24]:
def extrage_careu(original_img, cropped_mask_img):
    # image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    # image_m_blur = cv.medianBlur(image,3)
    # image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 5) 
    # image_sharpened = cv.addWeighted(image_m_blur, 1.2, image_g_blur, -0.8, 0)
    # #show_image('image_sharpened',image_sharpened)
    # _, thresh = cv.threshold(image_sharpened, 30, 255, cv.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(cropped_mask_img, kernel)
    #show_image('image_thresholded',thresh)

    edges =  cv.Canny(thresh ,50,400)
    cv.imshow("canny",edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 504
    height = 504
    
    # image_copy = cv.cvtColor(original_img.copy(),cv.COLOR_GRAY2BGR)
    image_copy = original_img.copy()
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),1)
    # cv.imshow("detected corners",image_copy)

    puzzle = np.array([top_left,top_right,bottom_right,bottom_left], dtype = "float32")
    destination_of_puzzle = np.array([[0,0],[width,0],[width,height],[0,height]], dtype = "float32")

    M = cv.getPerspectiveTransform(puzzle,destination_of_puzzle)

    result = cv.warpPerspective(original_img, M, (width, height))
    # result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)
    
    return result

In [ ]:
def preprocess(img):
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

    image_bi_blur = cv.bilateralFilter(gray, 15, 75, 75)

    # threshAdap = cv.adaptiveThreshold(gray, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 5, 2)
    T, threshOtsu = cv.threshold(image_bi_blur, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    # _, threshClassis1 = cv.threshold(crop_img1, 99, 255, cv.THRESH_BINARY_INV)
    # _, threshClassis1 = cv.threshold(gray, 95, 255, cv.THRESH_BINARY)
    
    return threshOtsu

In [ ]:
img1_r=cv.resize(img1,(0,0),fx=0.20,fy=0.20)
img2_r=cv.resize(img2,(0,0),fx=0.20,fy=0.20)

crop_img1 = crop_image_hsv(img1_r)
# cv.imshow("crop_img1",crop_img1)

careu_img1 = extrage_careu(original_img=img1_r,cropped_mask_img=crop_img1)
# cv.imshow("careu_img",careu_img1)

preproc_careu1 = preprocess(careu_img1)
# cv.imshow("preproc_careu1",preproc_careu1)

crop_img2 = crop_image_hsv(img2_r)
# cv.imshow("crop_img2",crop_img2)
careu_img2 = extrage_careu(original_img=img2_r,cropped_mask_img=crop_img2)
preproc_careu2 = preprocess(careu_img2)
# cv.imshow("careu_img",careu_img2)
cv.imshow("preproc_careu2",preproc_careu2)
# pre2 = preprocess(img2_r)
# crop_img2 = extrage_careu(original_img=img2_r,cropped_mask_img=pre2)
# cv.imshow("a",crop_img)
# # threshAdap = cv.adaptiveThreshold(crop_img, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 5, 2)
# # T, threshOtsu = cv.threshold(crop_img, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
# _, threshClassis1 = cv.threshold(crop_img1, 99, 255, cv.THRESH_BINARY_INV)
# _, threshClassis2 = cv.threshold(crop_img2, 99, 255, cv.THRESH_BINARY_INV)
kernel = np.ones((3,3),np.uint8)
# kernel = np.ones((4,4),np.uint8)
result = preproc_careu2 - preproc_careu1
result = cv.morphologyEx(result, cv.MORPH_OPEN, kernel)
# # cv.imshow("threshClassis",threshClassis)
# crop_img = extrage_careu(threshClassis)
# # cv.imshow("crop_img",crop_img)
cv.imshow("result",result)
# cv.imshow("2",threshClassis2)
# cv.imshow("threshAdap",a)
cv.waitKey(0)
cv.destroyAllWindows()